In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df= pd.read_csv('./DATA/Supervised_dataset.csv')

In [3]:
df.head()

,SpO2,HR,AccX,AccY,AccZ,Temp,EDA,Annotation,Subject,Age,Gender,Height_cm,Weight_kg,Annotation_Numeric
0,96.999969,89.000763,0.760015,-0.869990,-0.100004,30.100043,0.083039,Relax,Subject1,30,M,177,94,0
1,96.999969,88.000137,0.750005,-0.869990,-0.110031,30.100043,0.081028,Relax,Subject1,30,M,177,94,0
2,96.999969,87.000443,0.750005,-0.869990,-0.110031,30.100043,0.083039,Relax,Subject1,30,M,177,94,0
3,96.999969,87.000443,0.750005,-0.879993,-0.100004,30.100043,0.083039,Relax,Subject1,30,M,177,94,0
4,96.999969,87.000443,0.760015,-0.859987,-0.100004,30.100043,0.081028,Relax,Subject1,30,M,177,94,0
